In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pyecharts

In [2]:
df = pd.read_csv(r"D:\data\time_series_covid_19_confirmed.csv")

In [3]:
df.head()

Province/State Country/Region      Lat     Long  1/22/20  1/23/20  1/24/20  \
0            NaN    Afghanistan  33.0000  65.0000        0        0        0   
1            NaN        Albania  41.1533  20.1683        0        0        0   
2            NaN        Algeria  28.0339   1.6596        0        0        0   
3            NaN        Andorra  42.5063   1.5218        0        0        0   
4            NaN         Angola -11.2027  17.8739        0        0        0   

   1/25/20  1/26/20  1/27/20  ...  4/3/20  4/4/20  4/5/20  4/6/20  4/7/20  \
0        0        0        0  ...     281     299     349     367     423   
1        0        0        0  ...     304     333     361     377     383   
2        0        0        0  ...    1171    1251    1320    1423    1468   
3        0        0        0  ...     439     466     501     525     545   
4        0        0        0  ...       8      10      14      16      17   

   4/8/20  4/9/20  4/10/20  4/11/20  4/12/20  
0     444     484      521      555      607  
1     400     409      416      433      446  
2    1572    1666     1761     1825     1914  
3     564     583      601      601      638  
4      19      19       19       19       19  

[5 rows x 86 columns]

In [5]:
df.shape

(264, 86)

In [6]:
df.isnull().sum() #检查缺失值

Province/State    182
Country/Region      0
Lat                 0
Long                0
1/22/20             0
                 ... 
4/8/20              0
4/9/20              0
4/10/20             0
4/11/20             0
4/12/20             0
Length: 86, dtype: int64

In [7]:
df.isnull().sum().value_counts() #检查缺失值

0      85
182     1
dtype: int64

In [8]:
df.drop(columns="Province/State",inplace=True) #删除缺失值

In [9]:
df.duplicated().value_counts() #检查重复值

False    264
dtype: int64

In [10]:
gp1 = df.groupby("Country/Region").sum() #有些国家分州省统计的数据，依据国家进行分组加和，以国家作为最小统计单位
gp1.head()

Lat     Long  1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  \
Country/Region                                                                  
Afghanistan     33.0000  65.0000        0        0        0        0        0   
Albania         41.1533  20.1683        0        0        0        0        0   
Algeria         28.0339   1.6596        0        0        0        0        0   
Andorra         42.5063   1.5218        0        0        0        0        0   
Angola         -11.2027  17.8739        0        0        0        0        0   

                1/27/20  1/28/20  1/29/20  ...  4/3/20  4/4/20  4/5/20  \
Country/Region                             ...                           
Afghanistan           0        0        0  ...     281     299     349   
Albania               0        0        0  ...     304     333     361   
Algeria               0        0        0  ...    1171    1251    1320   
Andorra               0        0        0  ...     439     466     501   
Angola                0        0        0  ...       8      10      14   

                4/6/20  4/7/20  4/8/20  4/9/20  4/10/20  4/11/20  4/12/20  
Country/Region                                                             
Afghanistan        367     423     444     484      521      555      607  
Albania            377     383     400     409      416      433      446  
Algeria           1423    1468    1572    1666     1761     1825     1914  
Andorra            525     545     564     583      601      601      638  
Angola              16      17      19      19       19       19       19  

[5 rows x 84 columns]

In [11]:
gp2 = gp1.iloc[:,2:] #提取每天每个国家的确诊数据
gp2.head(2)

1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  1/27/20  1/28/20  \
Country/Region                                                                  
Afghanistan           0        0        0        0        0        0        0   
Albania               0        0        0        0        0        0        0   

                1/29/20  1/30/20  1/31/20  ...  4/3/20  4/4/20  4/5/20  \
Country/Region                             ...                           
Afghanistan           0        0        0  ...     281     299     349   
Albania               0        0        0  ...     304     333     361   

                4/6/20  4/7/20  4/8/20  4/9/20  4/10/20  4/11/20  4/12/20  
Country/Region                                                             
Afghanistan        367     423     444     484      521      555      607  
Albania            377     383     400     409      416      433      446  

[2 rows x 82 columns]

In [12]:
gp2.iloc[0].sum()

6065

In [13]:
gp2.apply(sum,axis=1) #计算每个国家的累积确诊人数

Country/Region
Afghanistan            6065
Albania                6630
Algeria               22032
Andorra                9068
Angola                  224
                      ...  
West Bank and Gaza     4164
Western Sahara           34
Yemen                     3
Zambia                  665
Zimbabwe                181
Length: 185, dtype: int64

In [14]:
country = gp2.iloc[:,-1].sort_values(ascending = False)[:10]  #按照累计确诊人数进行降序排序
country

Country/Region
US                555313
Spain             166831
Italy             156363
France            133670
Germany           127854
United Kingdom     85206
China              83134
Iran               71686
Turkey             56956
Belgium            29647
Name: 4/12/20, dtype: int64

In [15]:
country.index

Index(['US', 'Spain', 'Italy', 'France', 'Germany', 'United Kingdom', 'China',
       'Iran', 'Turkey', 'Belgium'],
      dtype='object', name='Country/Region')

In [16]:
gp3 = gp2[gp2.index.isin(country.index.tolist())] #从gp2中提取确诊人数前十的国家
gp3

1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  1/27/20  1/28/20  \
Country/Region                                                                  
Belgium               0        0        0        0        0        0        0   
China               548      643      920     1406     2075     2877     5509   
France                0        0        2        3        3        3        4   
Germany               0        0        0        0        0        1        4   
Iran                  0        0        0        0        0        0        0   
Italy                 0        0        0        0        0        0        0   
Spain                 0        0        0        0        0        0        0   
Turkey                0        0        0        0        0        0        0   
US                    1        1        2        2        5        5        5   
United Kingdom        0        0        0        0        0        0        0   

                1/29/20  1/30/20  1/31/20  ...  4/3/20  4/4/20  4/5/20  \
Country/Region                             ...                           
Belgium               0        0        0  ...   16770   18431   19691   
China              6087     8141     9802  ...   82511   82543   82602   
France                5        5        5  ...   65202   90848   93773   
Germany               4        4        5  ...   91159   96092  100123   
Iran                  0        0        0  ...   53183   55743   58226   
Italy                 0        0        2  ...  119827  124632  128948   
Spain                 0        0        0  ...  119199  126168  131646   
Turkey                0        0        0  ...   20921   23934   27069   
US                    5        5        7  ...  275586  308853  337072   
United Kingdom        0        0        2  ...   38689   42477   48436   

                4/6/20  4/7/20  4/8/20  4/9/20  4/10/20  4/11/20  4/12/20  
Country/Region                                                             
Belgium          20814   22194   23403   24983    26667    28018    29647  
China            82665   82718   82809   82883    82941    83014    83134  
France           98963  110065  113959  118781   125931   130727   133670  
Germany         103374  107663  113296  118181   122171   124908   127854  
Iran             60500   62589   64586   66220    68192    70029    71686  
Italy           132547  135586  139422  143626   147577   152271   156363  
Spain           136675  141942  148220  153222   158273   163027   166831  
Turkey           30217   34109   38226   42282    47029    52167    56956  
US              366667  396223  429052  461437   496535   526396   555313  
United Kingdom   52279   55949   61474   65872    74605    79874    85206  

[10 rows x 82 columns]

In [17]:
index = pd.Series(pd.date_range("1/22/20","4/12/20").strftime('%Y-%m-%d'))

In [18]:
index[index.values == index[0]].index

Int64Index([0], dtype='int64')

In [20]:
from pyecharts import options as opts
from pyecharts.charts import Line, Timeline

tl = Timeline()
for i in range(45,82) :
    line = (
        Line()
        .add_xaxis(index.values.tolist()[45:])
        .add_yaxis('France',gp3.loc['France'].values.tolist()[45:i+1],is_symbol_show = False)
        .add_yaxis('Germany',gp3.loc['Germany'].values.tolist()[45:i+1],is_symbol_show = False)
        .add_yaxis( 'Italy',gp3.loc[ 'Italy'].values.tolist()[45:i+1],is_symbol_show = False)
        .add_yaxis('Spain',gp3.loc['Spain'].values.tolist()[45:i+1],is_symbol_show = False)
        .add_yaxis('US',gp3.loc['US'].values.tolist()[45:i+1],is_symbol_show = False)
        .set_global_opts(title_opts=opts.TitleOpts(title="主要国家疫情折线图"))
            )
    tl.add_schema(play_interval = 200,is_loop_play = False).add(line, "{}".format(i)) #设置间隔时间200毫秒，设置不循环播放
# tl.render("每天疫情确诊情况.html")
tl.render_notebook()